In [1]:
import torch
import torch.nn as nn
import numpy as np
import scipy
import librosa
import youtube_dl
import os
import soundfile as sf
import musdb
from IPython.display import Audio, display

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cpu


In [2]:
%cd ../models/Conditioned-Source-Separation-LaSAFT

/Users/arron/code/PlexedLive/mixs/models/Conditioned-Source-Separation-LaSAFT


In [3]:
from lasaft.source_separation.conditioned.cunet.models.dcun_tfc_gpocm_lasaft import DCUN_TFC_GPoCM_LaSAFT_Framework

args = {}

# FFT params
args['n_fft'] = 2048
args['hop_length'] = 1024
args['num_frame'] = 128

# SVS Framework
args['spec_type'] = 'complex'
args['spec_est_mode'] = 'mapping'

# Other Hyperparams
args['optimizer'] = 'adam'
args['lr'] = 0.001
args['dev_mode'] = False
args['train_loss'] = 'spec_mse'
args['val_loss'] = 'raw_l1'

# DenseNet Hyperparams

args ['n_blocks'] = 7
args ['input_channels'] = 4
args ['internal_channels'] = 24
args ['first_conv_activation'] = 'relu'
args ['last_activation'] = 'identity'
args ['t_down_layers'] = None
args ['f_down_layers'] = None
args ['tif_init_mode'] = None

# TFC_TDF Block's Hyperparams
args['n_internal_layers'] =5
args['kernel_size_t'] = 3
args['kernel_size_f'] = 3
args['tfc_tdf_activation'] = 'relu'
args['bn_factor'] = 16
args['min_bn_units'] = 16
args['tfc_tdf_bias'] = True
args['num_tdfs'] = 6
args['dk'] = 32

args['control_vector_type'] = 'embedding'
args['control_input_dim'] = 4
args['embedding_dim'] = 32
args['condition_to'] = 'decoder'

args['control_n_layer'] = 4
args['control_type'] = 'dense'
args['pocm_type'] = 'matmul'
args['pocm_norm'] = 'batch_norm'


model = DCUN_TFC_GPoCM_LaSAFT_Framework(**args)

In [4]:
model = model.load_from_checkpoint('pretrained/gpocm_lasaft.ckpt')

In [5]:
!pwd

/Users/arron/code/PlexedLive/mixs/models/Conditioned-Source-Separation-LaSAFT


In [6]:
%cd ../../raw_data/musdb18/samples/test

/Users/arron/code/PlexedLive/mixs/raw_data/musdb18/samples/test


In [7]:
def separate_all(audio):
    '''
    you can add or remove what ever you want in this loop
    using the keys, vocals, drums, bass and other
    you can also combine them 
    '''
    
    print('vocals')
    vocals = model.separate_track(audio, 'vocals') 

    print('drums')
    drums = model.separate_track(audio, 'drums') 


    print('bass')
    bass = model.separate_track(audio, 'bass') 


    print('other')
    other = model.separate_track(audio, 'other') 

    
    return vocals, drums, bass, other

In [9]:
%%time
# path = '/content/drive/MyDrive/Musdbsample'
songs = [f for f in os.listdir() if not f.startswith('.')]
songs = sorted(songs)
#Sorted songs list to match songs_DB
estimates_dict = {}
songs_DB = musdb.DB(root='../../samples', subsets='test')
rate = 44100

for index, song in enumerate(songs):
    '''
    This will loop through all songs split and output the results into a dict
    NOTE - librosa is only to get the rate of the song
    '''  
    estimates_dict[song] = {}
    estimates_dict[song]['vocals'], estimates_dict[song]['drums'], estimates_dict[song]['bass'], estimates_dict[song]['other'] =\
    separate_all(songs_DB[index].audio)
    print(f'{song} completed')
    print(estimates_dict[song])

vocals
drums
bass
other
AM Contra - Heart Peripheral.stem.mp4 completed
{'vocals': array([[ 6.10351562e-05,  6.10351562e-05],
       [-2.74658203e-04, -5.49316406e-04],
       [-4.88281250e-04, -9.15527344e-05],
       ...,
       [-6.10351562e-05,  0.00000000e+00],
       [-2.74658203e-04, -4.57763672e-04],
       [ 4.27246094e-04,  3.35693359e-04]]), 'drums': array([[-5.49316406e-04, -2.13623047e-04],
       [-5.18798828e-04,  0.00000000e+00],
       [-3.96728516e-04, -2.74658203e-04],
       ...,
       [-5.49316406e-04, -5.49316406e-04],
       [-4.88281250e-04, -6.40869141e-04],
       [-9.15527344e-05,  3.05175781e-05]]), 'bass': array([[-4.57763672e-04, -4.57763672e-04],
       [ 9.15527344e-05,  1.22070312e-04],
       [-3.96728516e-04, -3.66210938e-04],
       ...,
       [-2.44140625e-04, -9.15527344e-05],
       [-3.66210938e-04, -9.15527344e-05],
       [-2.13623047e-04, -1.22070312e-04]]), 'other': array([[ 1.22070312e-04,  1.83105469e-04],
       [-3.96728516e-04, -3.6621

KeyboardInterrupt: 